In [1]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

startDate = '20201106'
endDate = '20201123'


readPath = r'\\192.168.10.34\random_backup\Kevin_zhenyu\orderLog\equityTradeLogs'
dataPathLs = np.array(glob.glob(os.path.join(readPath, 'speedCompare***.csv')))
dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
rawOrderLog = []
for thisDate, thisPath in zip(dateLs, dataPathLs):
    data = pd.read_csv(thisPath)
    data = data.rename(columns={'mdClockAtArrival': 'caamd'})
    rawOrderLog += [data]
rawOrderLog = pd.concat(rawOrderLog, sort=False)

for col in ['clockAtArrival', 'caamd', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
            'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
    rawOrderLog[col] = rawOrderLog[col].fillna(0).astype('int64')   
rawOrderLog = rawOrderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

rawOrderLog = rawOrderLog[rawOrderLog["secid"] >= 1000000]

display('There are accounts with duplicated ticks:')
display(rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep=False)]\
.groupby(['date', 'colo', 'accCode'])['ars'].size())
rawOrderLog = rawOrderLog.drop_duplicates(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep='first')

display('There are ticks with orderDirection 0')
display(rawOrderLog[rawOrderLog['orderDirection'] == 0][['date', 'colo', 'accCode', \
            'secid', 'vai', 'updateType', 'sdd', 'orderDirection', 'absOrderSize', 'internalId', 'orderId']])

# assert(rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
#                                                    .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
#                                                                 'vai', 'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
# if rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
#                                                    .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
#                                                                 'absOrderSize', 'internalId'], keep=False)].shape[0] != 0:
#     assert(rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
#                                                    .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
#                                                                 'absOrderSize', 'internalId'], keep=False)]['accCode'].unique() == [8856])
#     print('8856 has wrong internalId')
#     assert(sum(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
#             'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() != 1) == 0)
#     if rawOrderLog[(rawOrderLog['accCode'] != 8856) & (rawOrderLog['vai'] == -1)].shape[0] != 0:
#         print(rawOrderLog[(rawOrderLog['accCode'] != 8856) & (rawOrderLog['vai'] == -1)][['date', 'colo', 'accCode', \
#             'secid', 'vai', 'updateType', 'sdd', 'orderDirection', 'absOrderSize', 'internalId', 'orderId']])   
# else:
#     print('we can use internalId in 8856')
#     assert(sum(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
#             'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() != 1) == 0)   

assert(rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
                                                   .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                'vai', 'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                   .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
assert(sum(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() != 1) == 0)  

r2 = rawOrderLog[(rawOrderLog['accCode'] != 8856) & (rawOrderLog['orderDirection'] != 0)]
r1 = rawOrderLog[(rawOrderLog['accCode'] == 8856) & (rawOrderLog['orderDirection'] != 0)]
r1['test'] = r1.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize']).grouper.group_info[0]
r1 = r1.sort_values(by=['test', 'clockAtArrival'])
r1.loc[r1['updateType'] != 0, 'vai'] = np.nan
r1['vai'] = r1.groupby('test')['vai'].ffill()
r2['test'] = r2.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
r2 = r2.sort_values(by=['test', 'clockAtArrival'])
r2.loc[r2['updateType'] != 0, 'vai'] = np.nan
r2['vai'] = r2.groupby('test')['vai'].ffill()
assert(sum(r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
orderLog = pd.concat([r1, r2])
del r1
del r2    

orderLog['clock'] = orderLog['clockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
orderLog['broker'] = orderLog['accCode'] // 100
orderLog["broker"] = np.where(orderLog["accCode"].astype(str).apply(lambda x: len(x) == 6), orderLog['accCode'] // 10000, orderLog["broker"])
orderLog['colo_broker'] = orderLog['colo'].str[:2] + '_' + orderLog['broker'].astype('str')
orderLog['order'] = orderLog.groupby(['date', 'accCode', 'secid', 'vai']).grouper.group_info[0]
orderLog['group'] = orderLog.groupby(['date', 'secid', 'vai']).grouper.group_info[0]
orderLog['startClock'] = orderLog.groupby(['order'])['clockAtArrival'].transform('first')
orderLog['duration'] = orderLog['clockAtArrival'] - orderLog['startClock']
orderLog['orderPrice'] = orderLog['orderPrice'].apply(lambda x: round(x, 2))
orderLog['tradePrice'] = orderLog['tradePrice'].apply(lambda x: round(x, 2))
orderLog['orderDirection1'] = np.where(orderLog["orderDirection"] == -2, -1, np.where(
    orderLog["orderDirection"] == 2, 1, orderLog["orderDirection"]))
orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')
# rawOrderLog = rawOrderLog[rawOrderLog['ars'] % 10 == 1]


### Assertion 1:  make sure same direction in same date, secid, vai
print('=======================================================================================')
print('1. same date, secid, vai: same direction')
orderLog['directNum'] = orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].transform('nunique')
if len(orderLog[orderLog['directNum'] != 1]) > 0:
    print('opposite direction for same date, same secid, same vai')
    display(orderLog[(orderLog['directNum'] != 1) & (orderLog['updateType'] == 0)].groupby(['accCode'])['orderDirection'].size())
    orderLog = orderLog[orderLog['directNum'] == 1]

assert((orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].nunique() == 1).all() == True)

## Assertion 2:  make sure each account, secid, vai only has one insertion
print('=======================================================================================')
print('2. same date, secid, vai, accCode: one insertion')
a = orderLog[orderLog['updateType'] == 0].groupby(['date', 'colo', 'accCode', 'secid', 'vai', 'order'])['clockAtArrival'].count()
if len(a[a > 1]) > 0:
    print('more than one insertion at same time')
    a = a[a>1].reset_index()
    display(a)
    display(a.groupby(['accCode'])['date'].size())
    orderLog = orderLog[~(orderLog['order'].isin(a['order'].unique()))]

orderLog['isMsg'] = np.where(orderLog['updateType'] == 0, 
                             np.where(orderLog['mse'] == 100, 1, 0), np.nan)
orderLog['isMsg'] = orderLog.groupby(['order'])['isMsg'].ffill()

placeSZE = orderLog[(orderLog['secid'] >= 2000000) & (orderLog['updateType'] == 0)]
print('%.2f%% SZE orders triggered by msg data'%(placeSZE[placeSZE['isMsg'] == 1].shape[0]/placeSZE.shape[0]*100))


### Assertion 3:  check IPO stocks selling status
print('=======================================================================================')
print('3. IPO stocks selling (ars = 301, 302)')
if orderLog[orderLog['ars'].isin([301, 302])].shape[0] != 0:
    kk = orderLog[orderLog['ars'].isin([301, 302])]
    print(kk)
    try:
        assert(kk[kk['orderDirection1'] == 1].shape[0] == 0)
        print('we only sell, never buy')
    except:
        print('There are IPO buy side orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk[kk['orderDirection1'] == 1])
    kk1 = kk[kk['updateType'] == 0]
    kk1 = kk1.sort_values(by=['accCode', 'secid','clockAtArrival'])
    kk1['diff'] = kk1.groupby(['accCode', 'secid'])['clockAtArrival'].apply(lambda x: x-x.shift(1))
    kk1['diff'] = kk1['diff'].fillna(0)
    try:
        assert(kk1[kk1['diff'] < 10e6].shape[0] == 0)
        print('for each stock in the same account, there is no insertion within 10 seconds of the previous insertion')
    except:
        print('There are insertion within 10 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk1[kk1['diff'] < 10e6])
    kk2 = kk[(kk['updateType'] == 1)]
    try:
        assert(kk2[kk2['duration'] < 3e6].shape[0] == 0)
        print('for each stock in the same account, the cancellation of an order happens more than 3 seconds after the insertion')
    except:
        print('There are cancellation within 3 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk2[kk2['duration'] < 3e6])


### Assertion 4: check updateType == 7 orders, make sure updateType == 7 orders < 20 per account, < 100 in total
print('=======================================================================================')
print('4. updateType 7 orders')
if orderLog[orderLog['updateType'] == 7].shape[0] != 0:
    assert(orderLog[orderLog['updateType'] == 7].groupby(['date', 'accCode'])['order'].nunique().max() < 20)
    assert(orderLog[orderLog['updateType'] == 7].groupby(['date', 'accCode'])['order'].nunique().sum() < 100)

### Assertion 5: check updateType == 6 orders, make sure updateType == 6 orders < 5% per account
print('=======================================================================================')
print('5. updateType 6 orders')
k1 = orderLog[orderLog['updateType'] == 6].groupby(['date', 'accCode'])['order'].nunique().reset_index()
k2 = orderLog.groupby(['date', 'accCode'])['order'].nunique().reset_index()
k = pd.merge(k1, k2, on=['date', 'accCode'], how='left')
k['prob'] = k['order_x']/k['order_y']
try:
    assert(sum(k['prob'] >= 0.05) == 0)
except:
    print('There are accounts with more than 5% updateType 6 orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    print(k[k['prob'] >= 0.05])

### Assertion 6: check CYB orders, make sure all CYB stocks have absOrderSize < 30w
print('=======================================================================================')
print('6. CYB stocks order size < 30w')
try:
    cyb = orderLog[(orderLog['secid'] >= 2300000) & (orderLog['updateType'] == 0)].groupby('date')['absOrderSize'].max().reset_index()
    assert(cyb[cyb['absOrderSize'] > 300000].shape[0] == 0)
except:
    print('CYB stocks total absOrderSize >= 30w!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    
     
### Assertion 7:  make sure there is no unexpected updateType 
print('=======================================================================================')
print('7. unexpected updateType')
def getTuple(x):
    return tuple(i for i in x)

checkLog = orderLog[~((orderLog['updateType'] == 4) & (orderLog.groupby(['order'])['updateType'].shift(-1) == 4))]
checkLog = checkLog.groupby(['order'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
checkLog['status'] = np.where(checkLog['updateType'].isin([(0, 2, 4), (0, 2, 1, 4), (0, 2, 1, 2, 4), (0, 2, 4, 1, 4), (0, 4), (0, 1, 4), (0, 4, 1, 4), (0, 2, 2, 4), (0, 4, 2, 4), (0, 2, 2, 1, 4), (0, 2, 2, 4, 1, 4)]),0,
                     np.where(checkLog['updateType'].isin([(0, 2, 4, 1, 3), (0, 2, 4, 1, 4, 3), (0, 2, 1, 4, 3), (0, 4, 1, 3), (0, 1, 4, 3),
                                                               (0, 2, 2, 4, 1, 3), (0, 2, 2, 4, 1, 4, 3), (0, 2, 2, 1, 4, 3), (0, 4, 2, 4, 1, 3),
                                                               (0, 4, 2, 1, 3), (0, 4, 1, 4, 3), (0, 4, 1)]), 1,
                     np.where(checkLog['updateType'].isin([(0, 2, 1, 3), (0, 2, 2, 1, 3), (0, 2, 3), (0, 3), (0, 1, 3), (0, ), (0, 2), (0, 2, 1), (0, 2, 2)]), 2, 3)))

orderLog = pd.merge(orderLog, checkLog[['order', 'status']], how='left', on=['order'], validate='many_to_one')
orderLog = orderLog[orderLog['status'].isin([0, 1, 2])].reset_index(drop=True)

### Assertion 8:  make sure status==0 got all traded
print('=======================================================================================')
print('8. status == 0: all traded')
a = orderLog[orderLog['status'] == 0]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
print('in total trade, any fill != total cases')
display(a[a['filled'] != a['total']])
if a[a['filled'] != a['total']].shape[0] > 0:
    removeOrderLs = a[a['filled'] != a['total']]['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
### Assertion 9:  make sure status==1 got partial traded
print('=======================================================================================')
print('9. status == 1: partial traded')
a = orderLog[orderLog['status'] == 1]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
print('in partial trade, any fill >= total or fill is 0 cases for updateType 4')
display(a[(a['filled'] >= a['total']) | (a['filled'] == 0)])
if a[(a['filled'] >= a['total']) | (a['filled'] == 0)].shape[0] > 0:
    removeOrderLs = a[(a['filled'] >= a['total']) | (a['filled'] == 0)]['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
### Assertion 10: make sure no cancellation within 1 sec
print('=======================================================================================')
print('10. no cancellation within 1 sec')
a = orderLog[(orderLog['updateType'] == 1) & (orderLog['duration'] < 1e6)]
print('any cancellation within 1 sec')
display(a)
if a.shape[0] > 0:
    removeOrderLs = a['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


### Assertion 11: make sure no order has shares > 80w or notional > 800w
print('=======================================================================================')
print('11. Orders with size > 80w or notional > 800w')
orderLog['orderNtl'] = orderLog['absOrderSize'] * orderLog['orderPrice']
if orderLog[orderLog['absOrderSize'] > 800000].shape[0] > 0:
    print('some order quantity are > 80w')
    print(orderLog[orderLog['absOrderSize'] > 800000].groupby(['colo', 'accCode'])['order'].nunique())
    display(orderLog[orderLog['absOrderSize'] > 800000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                         'orderNtl', 'orderDirection', 'clock', 'order']])
            
if orderLog[orderLog['orderNtl'] > 8000000].shape[0] > 0:
    print('some order ntl are > 800w')
    print(orderLog[orderLog['orderNtl'] > 8000000].groupby(['colo', 'accCode'])['order'].nunique())
    display(orderLog[orderLog['orderNtl'] > 8000000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                      'orderNtl', 'orderDirection', 'clock', 'order', "updateType", 
                                                      "tradePrice", "absOrderSizeCumFilled", "absFilledThisUpdate"]])

removeOrderLs = list(set(orderLog[orderLog['absOrderSize'] > 800000]['order'].unique()) | set(orderLog[orderLog['orderNtl'] > 8000000]['order'].unique()))
orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

orderLog["mrsb90"] = orderLog.groupby(['order'])['mrsb90'].transform('first')
orderLog["mrss90"] = orderLog.groupby(['order'])['mrss90'].transform('first')
orderLog["aaa"] = orderLog.groupby(['order'])['aaa'].transform('first')

orderLog['m1'] = orderLog['mrstaat'].apply(lambda x: x - (x // 10000) * 10000)
orderLog['m2'] = orderLog['mrstauc'].apply(lambda x: x - (x // 10000) * 10000)
if orderLog[orderLog['mrsb90'] == '-'].shape[0] != 0:
    display(orderLog[orderLog['mrsb90'] == '-'])
orderLog = orderLog[orderLog['mrsb90'] != '-']
orderLog['mrsb90'] = orderLog['mrsb90'].astype(float)
if orderLog[orderLog['aaa'] == '-'].shape[0] != 0:
    display(orderLog[orderLog['aaa'] == '-'])
orderLog = orderLog[orderLog['aaa'] != '-']
orderLog['aaa'] = orderLog['aaa'].astype(float)
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstauc'] = \
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm2']

orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstaat'] = \
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm1']

orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstauc'] = \
orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm2']

orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstaat'] = \
orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm1']    

orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')
orderLog['orderNtl'] = orderLog['orderPrice'] * orderLog['absOrderSize']
orderLog['tradeNtl'] = np.where(orderLog['updateType'] == 4, orderLog['tradePrice']*orderLog['absFilledThisUpdate'], 0)
orderLog["mrstaat"] = orderLog.groupby(['order'])['mrstaat'].transform('first')
orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')
orderLog["mrstauc"] = orderLog.groupby(['order'])['mrstauc'].transform('first')
orderLog = orderLog[~orderLog['ars'].isnull()]
orderLog['sta'] = np.where(orderLog['mrstaat'] == 1000, '1. staone', np.where(
orderLog['mrstaat'] == 3000, '2. statwo', np.where(
orderLog['mrstaat'].isin([11000, 13000]), '3. sta300', 'else')))
display(orderLog[orderLog['sta'] == 'else'][['date', 'secid', 'colo', 'accCode', 'updateType', 'vai', 'absOrderSize', 'orderDirection', 'mrstaat', 'mrstauc', 'sdd', 'ApplSeqNum', 'sta']])
orderLog = orderLog[orderLog['mrstaat'].isin([11000, 13000, 1000, 3000])]
display(orderLog[orderLog['updateType'] == 0].groupby(['mrstaat', 'mrstauc'])['date'].size())

C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (15,16,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (15,16,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=res

'There are accounts with duplicated ticks:'

date      colo      accCode
20201106  zs_88_04  892402     11272
          zs_96_02  9655        1450
          zt_88_02  897102     20282
20201112  zs_96_08  974101     27542
          zt_88_06  8971       25984
Name: ars, dtype: int64

'There are ticks with orderDirection 0'

,date,colo,accCode,secid,vai,updateType,sdd,orderDirection,absOrderSize,internalId,orderId
387349,20201106,zs_94_05,9471,2000049,-1,7,35262.0,0,0,-1.0,-1.000000e+00
402657,20201106,zs_96_08,9685,2000516,-1,1,48383.0,0,0,-1.0,7.221449e+17
407058,20201106,zs_54_01,5470,2000565,-1,7,40926.0,0,0,-1.0,-1.000000e+00
428703,20201106,zs_54_01,5470,2000731,-1,1,53406.0,0,0,-1.0,8.689083e+08
450878,20201106,zs_66_01,6634,2000920,-1,1,52100.0,0,0,-1.0,1.812811e+10
...,...,...,...,...,...,...,...,...,...,...,...
11155576,20201123,zs_52_06,5269,2300331,-1,1,39750.0,0,0,-1.0,8.887090e+05
11173817,20201123,zs_96_08,9741,2300416,-1,7,37225.0,0,0,-1.0,-1.000000e+00
11259177,20201123,zs_66_01,6631,2300677,-1,1,51116.0,0,0,-1.0,1.812810e+10
11274611,20201123,zs_66_01,6634,2300715,-1,1,52007.0,0,0,-1.0,1.812810e+10


...

11. Orders with size > 80w or notional > 800w
some order ntl are > 800w
colo      accCode
zt_52_05  527501     1
zt_88_06  8971       2
Name: order, dtype: int64


,date,accCode,secid,vai,absOrderSize,orderPrice,orderNtl,orderDirection,clock,order,updateType,tradePrice,absOrderSizeCumFilled,absFilledThisUpdate
3921068,20201111,8971,1600031,68524384.0,277800,28.8,8000640.0,1,2020-11-11 13:04:54.879842,854468,0,-1.0,0,0
3921069,20201111,8971,1600031,68524384.0,277800,28.8,8000640.0,1,2020-11-11 13:04:55.034330,854468,2,-1.0,0,0
3921070,20201111,8971,1600031,68524384.0,277800,28.8,8000640.0,1,2020-11-11 13:04:55.275614,854468,4,28.8,300,300
3921071,20201111,8971,1600031,68524384.0,277800,28.8,8000640.0,1,2020-11-11 13:04:55.275760,854468,4,28.8,406,106
3921072,20201111,8971,1600031,68524384.0,277800,28.8,8000640.0,1,2020-11-11 13:04:55.275903,854468,4,28.8,2406,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6807490,20201116,8971,1600585,19334231.0,141600,56.5,8000400.0,1,2020-11-16 11:02:10.974065,1571186,4,56.5,121000,100
6807491,20201116,8971,1600585,19334231.0,141600,56.5,8000400.0,1,2020-11-16 11:02:10.974422,1571186,4,56.5,121200,200
6807492,20201116,8971,1600585,19334231.0,141600,56.5,8000400.0,1,2020-11-16 11:02:10.974591,1571186,4,56.5,121300,100
6807493,20201116,8971,1600585,19334231.0,141600,56.5,8000400.0,1,2020-11-16 11:02:11.658279,1571186,1,-1.0,121300,0


C:\Users\win\Anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,date,secid,colo,accCode,updateType,vai,absOrderSize,orderDirection,mrstaat,mrstauc,sdd,ApplSeqNum,sta
709366,20201106,2300055,zt_88_02,8854,0,0.0,1400,-1,NaN,NaN,145644710.0,NaN,else
997476,20201106,2300903,zs_96_08,9685,0,302.0,300,-1,0.0,0.0,105157000.0,0.0,else
997477,20201106,2300903,zs_96_08,9685,2,302.0,300,-1,0.0,0.0,-1.0,0.0,else
997478,20201106,2300903,zs_96_08,9685,4,302.0,300,-1,0.0,0.0,-1.0,0.0,else
997479,20201106,2300903,zt_96_09,968501,0,302.0,300,-1,0.0,0.0,105157000.0,0.0,else
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035652,20201123,2003019,zs_96_08,9685,0,302.0,216,-1,0.0,0.0,134147260.0,0.0,else
11035653,20201123,2003019,zs_96_08,9685,2,302.0,216,-1,0.0,0.0,-1.0,0.0,else
11035654,20201123,2003019,zs_96_08,9685,4,302.0,216,-1,0.0,0.0,-1.0,0.0,else
11035655,20201123,2003019,zs_96_08,9685,4,302.0,216,-1,0.0,0.0,-1.0,0.0,else


mrstaat  mrstauc
1000.0   0.0         390620
3000.0   1000.0     1974326
         2000.0      138758
         3000.0       58696
11000.0  10000.0       7948
         20000.0        476
         30000.0        271
13000.0  11000.0     155017
         12000.0       7776
         13000.0       2482
         21000.0       2862
         22000.0       3162
         23000.0       1193
         31000.0       1153
         32000.0        998
         33000.0       1299
Name: date, dtype: int64

In [41]:
orderLog[orderLog['accCode'] == 5332]['colo'].unique()

array(['zs_52_06'], dtype=object)

In [68]:
rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
                                                   .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                'absOrderSize', 'internalId'], keep=False)]['accCode'].unique()

array([8856], dtype=int64)

In [62]:
orderLog[(orderLog['accCode'] == 8865) & (orderLog['updateType'] == 0)]

,Unnamed: 0,date,clockAtArrival,caamd,colo,accCode,secid,vai,sdd,orderDirection,absOrderSize,updateType,tradePrice,absFilledThisUpdate,absOrderSizeCumFilled,tradeId,orderSysId,aaa,internalId,ars,mse,mrsb300,mfe,mra100,session,hee,mt,sequenceNo,mrsb90,mrss90,ms,threadId,mta,mrstauc,orderPrice,finalState,mrrlma,l4tr,mrstaum,clock,cancellationPending,mrstaat,mrss300,underlyingIndex,ApplSeqNum,cfe,mrm,mrb100,orderId,mrmum,mrm25,test,broker,colo_broker,order,group,startClock,duration,orderDirection1,directNum,isMsg,status,orderNtl,m1,m2,exchange,tradeNtl,sta,sta_uc,server,server_account,tag
382993,152772,20201106,1604631577782927,1604631577782875,zs_88_04,8865,2000088,19522335.0,105938010.0,-1,500,0,-1.0,0,0,NaN,NaN,0.001994,716.0,1.0,100,-1,-1.0,686.0,0,0.000845,0.0,149863172.0,-0.003589,0.001994,10:59:37.781538,312802.0,-0.007564,1000.0,6.85,0.0,0.0,0.0,0.0,2020-11-06 10:59:37.782927,0.0,3000.0,-1.0,852,13069608.0,4860513.51,-0.007564,685.0,-1.0,NaN,NaN,104024,88,zs_88,113589,46276,1604631577782927,0,-1,1.0,1.0,0,3425.0,3000.0,1000.0,SZE,0.0,2. statwo,2. statwo_1000,zs8804,zs8804_8865,CSI1000
383000,152776,20201106,1604631578725175,1604631578725123,zs_88_04,8865,2000088,19525335.0,105938950.0,-1,3700,0,-1.0,0,0,NaN,NaN,0.002013,717.0,1.0,100,-1,-1.0,686.0,0,0.000845,0.0,149872031.0,-0.003610,0.002013,10:59:38.724538,312802.0,-0.007564,1000.0,6.85,0.0,0.0,0.0,0.0,2020-11-06 10:59:38.725175,0.0,3000.0,-1.0,852,13070362.0,4863938.51,-0.007564,685.0,-1.0,NaN,NaN,104032,88,zs_88,113590,46277,1604631578725175,0,-1,1.0,1.0,0,25345.0,3000.0,1000.0,SZE,0.0,2. statwo,2. statwo_1000,zs8804,zs8804_8865,CSI1000
383005,152781,20201106,1604631579325778,1604631579325727,zs_88_04,8865,2000088,19548735.0,105939550.0,-1,1000,0,-1.0,0,0,NaN,NaN,0.002286,718.0,1.0,100,-1,-1.0,686.0,0,0.000848,0.0,149877855.0,-0.003864,0.002286,10:59:39.324538,312802.0,-0.007564,1000.0,6.85,0.0,0.0,0.0,0.0,2020-11-06 10:59:39.325778,0.0,3000.0,-1.0,852,13070833.0,4889283.51,-0.007564,685.0,-1.0,NaN,NaN,104029,88,zs_88,113591,46278,1604631579325778,0,-1,1.0,1.0,0,6850.0,3000.0,1000.0,SZE,0.0,2. statwo,2. statwo_1000,zs8804,zs8804_8865,CSI1000
383008,152784,20201106,1604631579913007,1604631579912957,zs_88_04,8865,2000088,19550235.0,105940140.0,-1,600,0,-1.0,0,0,NaN,NaN,0.001626,719.0,1.0,100,-1,-1.0,686.0,0,0.000849,0.0,149889042.0,-0.003174,0.001626,10:59:39.911538,312802.0,-0.007564,1000.0,6.85,0.0,0.0,0.0,0.0,2020-11-06 10:59:39.913007,0.0,3000.0,-1.0,852,13071475.0,4896133.51,-0.007564,685.0,-1.0,NaN,NaN,104025,88,zs_88,113592,46279,1604631579913007,0,-1,1.0,1.0,0,4110.0,3000.0,1000.0,SZE,0.0,2. statwo,2. statwo_1000,zs8804,zs8804_8865,CSI1000
383011,152787,20201106,1604631580440442,1604631580440389,zs_88_04,8865,2000088,19550835.0,105940670.0,-1,300,0,-1.0,0,0,NaN,NaN,0.001699,720.0,1.0,100,-1,-1.0,686.0,0,0.000849,0.0,149897280.0,-0.003250,0.001699,10:59:40.439538,312802.0,-0.007564,1000.0,6.85,0.0,0.0,0.0,0.0,2020-11-06 10:59:40.440442,0.0,3000.0,-1.0,852,13071897.0,4900243.51,-0.007564,685.0,-1.0,NaN,NaN,104023,88,zs_88,113593,46280,1604631580440442,0,-1,1.0,1.0,0,2055.0,3000.0,1000.0,SZE,0.0,2. statwo,2. statwo_1000,zs8804,zs8804_8865,CSI1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11258418,726961,20201123,1606108048759076,1606108048759022,zs_88_04,8865,2300815,686300.0,130715810.0,1,100,0,-1.0,0,0,NaN,NaN,0.001062,1029.0,1.0,100,-1,-1.0,9857.0,0,0.000717,0.0,162577656.0,0.001062,-0.001497,13:07:28.757543,71777.0,0.001417,1000.0,98.57,0.0,0.0,0.0,0.0,2020-11-23 13:07:28.759076,0.0,3000.0,-1.0,852,14346947.0,2340443.86,0.001417,9853.0,-1.0,0.0,0.001225,2624979,88,zs_88,2636746,1446896,1606108048759076,0,1,1.0,1.0,2,9857.0,3000.0,1000.0,SZE,0.0,2. statwo,2. statwo_1000,zs8804,zs8804_8865,CSI1000
11258435,726974,2020

In [42]:
orderLog = orderLog[~orderLog['mrstauc'].isnull()]
orderLog['sta_uc'] = orderLog['sta'] + '_' + orderLog['mrstauc'].astype(int).astype(str)
orderLog["server"] = orderLog["colo"].apply(lambda x: x.split("_")[0] + x.split("_")[1] + x.split("_")[2])
orderLog["server_account"] = orderLog["server"] + '_' + orderLog['accCode'].astype('str')
checkLog = orderLog[orderLog['updateType'].isin([0, 3, 4])].reset_index(drop=True)
checkLog = checkLog[checkLog['date'] >= 20201117]
checkLog = checkLog[checkLog['duration'] <= 1e6]

checkLog["time"] = checkLog["clock"].apply(lambda x: x.strftime("%H%M%S")).astype(int)
# ol = checkLog[(checkLog["updateType"] == 0) & (~checkLog["ars"].isin([121, 221, 321, 131, 231, 331])) & (checkLog["time"]>= 93000)
#              & (checkLog["time"] < 93300)]["order"].unique()

sta_list = orderLog['sta_uc'].unique()

result = {}
for col in ['date', 'broker1', 'broker2', 'isFast']:
    result[col] = []
            
for i in sta_list:
    checkLog1 = checkLog[checkLog['sta_uc'] == i]
    checkLog1['accountNum'] = checkLog1.groupby(['date', 'secid', 'vai'])['server_account'].transform('nunique')
    checkLog1 = checkLog1[checkLog1['accountNum'] >= 2]
    checkLog1['cumFillSize'] = checkLog1.groupby(['order'])['absOrderSizeCumFilled'].transform('max')
    checkLog1['status'] = np.where(checkLog1['cumFillSize'] == 0, 2, 
                                    np.where(checkLog1['cumFillSize'] < checkLog1['absOrderSize'], 1, 0))
    checkLog1 = checkLog1.groupby(['group', 'order'])[['date', 'server_account', 'secid', 'vai', 'status']].first().reset_index()

    for exchg in ['SZE']:

        print(exchg)

        if exchg == 'SSE':
            exchangeLog = checkLog1[checkLog1['secid'] < 2000000]
        else:
            exchangeLog = checkLog1[checkLog1['secid'] >= 2000000]

        for group, groupData in exchangeLog.groupby(['group']):
            date = groupData['date'].values[0]
            brokerLs = groupData['server_account'].values
            if len(brokerLs) == 0:
                continue
            statusLs = groupData['status'].values
            ixLs = [i for i in range(len(brokerLs))]
            for k, broker1, status1 in zip(ixLs[:-1], brokerLs[:-1], statusLs[:-1]):
                for broker2, status2 in zip(brokerLs[k+1:], statusLs[k+1:]):
                    if broker1 != broker2:
                        result['date'] += [date, date]
                        result['broker1'] += [broker1, broker2]
                        result['broker2'] += [broker2, broker1]

                        if status1 < status2:
                            result['isFast'] += [1, 0]
                        elif status1 > status2:
                            result['isFast'] += [0, 1]
                        else:
                            result['isFast'] += [-1, -1]                

result = pd.DataFrame(result)
result = result[result['isFast'] != -1]
result['count'] = result.groupby(['broker1', 'broker2'])['isFast'].transform('count')
result['fasterSum'] = result.groupby(['broker1', 'broker2'])['isFast'].transform('sum')
result['faster'] = result['fasterSum']/result['count']


summary = pd.DataFrame()
summaryCols = []
countCols = []
fasterCols = []
for date in result['date'].unique():
    a = result[result['date'] == date].reset_index(drop=True)
    a['count'] = a.groupby(['broker1', 'broker2'])['isFast'].transform('count')
    a['fasterSum'] = a.groupby(['broker1', 'broker2'])['isFast'].transform('sum')
    a['faster'] = a['fasterSum']/a['count']
    a = a.groupby(['broker1', 'broker2'])[['count', 'faster']].mean().reset_index()
    a.columns = ['broker1', 'broker2', '%s count'%date, '%s faster'%date]
    summaryCols += ['%s count'%date, '%s faster'%date]
    countCols += ['%s count'%date]
    fasterCols += ['%s faster'%date]

    if summary.empty:
        summary = a.copy()
    else:
        summary = pd.merge(summary, a, how='outer', on=['broker1', 'broker2'], validate='one_to_one')

a = result.groupby(['broker1', 'broker2'])[['count', 'faster']].mean().reset_index()
a.columns = ['broker1', 'broker2', 'total count', 'total faster']
summary = pd.merge(summary, a, how='outer', on=['broker1', 'broker2'], validate='one_to_one')
summaryCols = ['total count', 'total faster'] + summaryCols


savePath = r'L:\orderLog\result\relativeSpeed'
a = summary.groupby(['broker1', 'broker2'])[['total count', 'total faster']].first().reset_index()
a.to_csv(os.path.join(savePath, 'relativeSpeedAccount_%s_%s_%s.csv'%(startDate, endDate, exchg)), index=False)




countCols += ['total count']
fasterCols += ['total faster']
summary['b1'] = summary['broker1'].str[7:11].astype(int) // 100
summary['b2'] = summary['broker2'].str[7:11].astype(int) // 100
#     summary = summary.groupby(['broker1', 'broker2'])[summaryCols].first()
for col in countCols:
    summary[col] = summary[col].fillna(0)
    summary[col] = summary[col].astype('int64')

for col in fasterCols:
    summary[col] = summary[col].fillna(-1)
    summary[col] = summary[col].apply(lambda x: '%.0f%%'%(x*100))
    summary[col] = np.where(summary[col] == '-100%', ' ', summary[col])

#     summary = summary.rename(columns={"broker1":"account1", "broker2":"account2"})
#     summary = summary.groupby(['account1', 'account2'])[summaryCols].first()
#     from IPython.display import display, HTML
#     display(HTML(summary.iloc[:, :2].to_html()))
#     summary["tt"]=summary[["broker1", "broker2"]].min(axis=1).astype(str) + ' - ' + summary[["broker1", "broker2"]].max(axis=1).astype(str)
#     summary = summary.sort_values(by=["tt", "broker1"])
#     s1 = summary.groupby("tt").first().reset_index(drop=True)
s1 = summary
display(s1[(s1["broker1"] == 'zs5206_5332')])

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


SZE
SZE
SZE
SZE
SZE
SZE
SZE
SZE
SZE
SZE
SZE
SZE
SZE
SZE
SZE
SZE


,broker1,broker2,20201117 count,20201117 faster,20201118 count,20201118 faster,20201119 count,20201119 faster,20201120 count,20201120 faster,20201123 count,20201123 faster,total count,total faster,b1,b2
81,zs5206_5332,zs5206_5328,16,38%,1,0%,0,,0,,0,,17,35%,53,53
82,zs5206_5332,zs5208_5229,154,1%,131,1%,152,1%,83,1%,161,7%,681,2%,53,52
83,zs5206_5332,zs5208_5276,293,6%,610,2%,318,4%,0,,413,13%,1634,6%,53,52
84,zs5206_5332,zs5208_5377,63,100%,68,99%,49,100%,25,100%,136,99%,341,99%,53,53
85,zs5206_5332,zs5208_5386,115,100%,115,100%,87,100%,48,100%,167,99%,532,100%,53,53
86,zs5206_5332,zs5209_5289,218,1%,323,1%,106,1%,84,1%,217,9%,948,3%,53,52
87,zs5206_5332,zs5209_5291,56,7%,98,4%,129,1%,63,6%,115,14%,461,6%,53,52
88,zs5206_5332,zs8804_8856,26,0%,48,2%,22,9%,8,12%,12,25%,116,6%,53,88
89,zs5206_5332,zs8804_8924,18,100%,17,100%,8,88%,4,100%,0,,47,98%,53,89
90,zs5206_5332,zs8804_896702,8,100%,7,100%,3,100%,3,100%,0,,21,100%,53,89


In [8]:
import pandas as pd
data = pd.read_csv(r'\\192.168.10.28\equityTradeLogs\speedCompare_20201218.csv')
data[(data['updateType'] == 0) & (data['accCode'] == 7293)].shape[0]

216

In [49]:
checkLog[checkLog['server_account'] == 'zs5209_5291']['orderDirection'].unique()

array([ 1, -1], dtype=int64)

In [48]:
display(s1[(s1["broker1"] == 'zs5206_5332') & (s1['broker2'].str[7:9] == '52')])

,broker1,broker2,20201117 count,20201117 faster,20201118 count,20201118 faster,20201119 count,20201119 faster,20201120 count,20201120 faster,20201123 count,20201123 faster,total count,total faster,b1,b2
82,zs5206_5332,zs5208_5229,154,1%,131,1%,152,1%,83,1%,161,7%,681,2%,53,52
83,zs5206_5332,zs5208_5276,293,6%,610,2%,318,4%,0,,413,13%,1634,6%,53,52
86,zs5206_5332,zs5209_5289,218,1%,323,1%,106,1%,84,1%,217,9%,948,3%,53,52
87,zs5206_5332,zs5209_5291,56,7%,98,4%,129,1%,63,6%,115,14%,461,6%,53,52


In [30]:
orderLog[(orderLog['accCode'].isin([8865, 8856])) & (orderLog['updateType'] == 0)].groupby(['tag', 'exchange', 'colo'])['accCode'].unique()

tag      exchange  colo    
CSI1000  SZE       zs_88_04    [8865]
IC       SZE       zs_88_04    [8856]
Name: accCode, dtype: object

In [29]:
orderLog['tag'] = 'IC'
orderLog.loc[orderLog['accCode'] == 8865, 'tag'] = 'CSI1000'

In [32]:
orderLog[(orderLog['accCode'].isin([8833, 8854, 8865, 8886, 8856])) & (orderLog['updateType'] == 0) & (orderLog['exchange'] == 'SZE')].groupby(['sta', 'colo', 'accCode'])['date'].size()

sta        colo      accCode
1. staone  zs_88_04  8856         909
                     8865        1036
           zt_88_02  8854        1839
           zt_88_03  8833        3632
           zt_88_06  8854        1887
2. statwo  zs_88_04  8856        3489
                     8865       21583
           zt_88_03  8833       14393
           zt_88_06  8854       18059
Name: date, dtype: int64

In [17]:
orderLog[(orderLog['sta'] == '1. staone') & (orderLog['exchange'] == 'SZE') & (orderLog['accCode'] == 8854) & (orderLog['colo'] == 'zt_88_06') & (orderLog['updateType'] == 0) & (orderLog['date'] == 20201109)]['secid'].min()

2000403

In [25]:
pd.set_option('max_columns', 200)
rawOrderLog[(rawOrderLog['vai'] == 3949903) & (rawOrderLog['updateType'] == 0) & (rawOrderLog['date'] == 20201109) & (rawOrderLog['secid'] == 2000049)]

,Unnamed: 0,date,clockAtArrival,caamd,colo,accCode,secid,vai,sdd,orderDirection,absOrderSize,updateType,tradePrice,absFilledThisUpdate,absOrderSizeCumFilled,tradeId,orderSysId,aaa,internalId,ars,mse,mrsb300,mfe,mra100,session,hee,mt,sequenceNo,mrsb90,mrss90,ms,threadId,mta,mrstauc,orderPrice,finalState,mrrlma,l4tr,mrstaum,clock,cancellationPending,mrstaat,mrss300,underlyingIndex,ApplSeqNum,cfe,mrm,mrb100,orderId,mrmum,mrm25
1408978,141022,20201109,1604898118889181,1604898118889153,zt_88_06,8854,2000049,3949903,130154860.0,-1,500,0,-1.0,0,0,NaN,NaN,-0.000989,43.0,2.0,100,-1,-1.0,5534.0,0,0.0002,0.0,92500654.0,0.000079,-0.000989,13:01:58.888068,262935.0,-0.000395,0.0,55.33,0.0,0.0,31301.0,0.0,1.604898e+15,0.0,1000.0,-1.0,852,16861537.0,164465.27,-0.000395,5529.0,-1.0,NaN,NaN


In [44]:
orderLog.loc[orderLog['accCode'] == 8886, 'tag'] = 'IC'

In [39]:
orderLog[(orderLog['updateType'] == 0) & (orderLog['accCode'] == 8886)]['exchange'].unique()

array(['SSE'], dtype=object)

In [41]:
set(orderLog[(orderLog['updateType'] == 0) & (orderLog['accCode'] == 8886) & (orderLog['date'] == 20201120)]['secid'].unique()) - set(IC['skey'].unique())

set()

In [18]:
import pymongo
import pandas as pd
import numpy as np
import pickle
import datetime
import time
import gzip
import lzma
import pytz

def DB(host, db_name, user, passwd):
    auth_db = db_name if user not in ('admin', 'root') else 'admin'
    url = 'mongodb://%s:%s@%s/?authSource=%s' % (user, passwd, host, auth_db)
    client = pymongo.MongoClient(url, maxPoolSize=None)
    db = client[db_name]
    return db

def read_memb_daily(db, name, start_date=None, end_date=None, skey=None, index_id=None, interval=None, col=None, return_sdi=True):
    collection = db[name]
    # Build projection
    prj = {'_id': 0}
    if col is not None:
        if return_sdi:
            col = ['skey', 'date', 'interval'] + col
        for col_name in col:
            prj[col_name] = 1

    # Build query
    query = {}
    if skey is not None:
        query['skey'] = {'$in': skey}
    if index_id is not None:
        query['index_id'] = {'$in': index_id}
    if interval is not None:
        query['interval'] = {'$in': interval}
    if start_date is not None:
        if end_date is not None:
            query['date'] = {'$gte': start_date, '$lte': end_date}
        else:
            query['date'] = {'$gte': start_date}
    elif end_date is not None:
        query['date'] = {'$lte': end_date}

    # Load data
    cur = collection.find(query, prj)
    df = pd.DataFrame.from_records(cur)
    if df.empty:
        df = pd.DataFrame()
    else:
        df = df.sort_values(by=['date', 'index_id', 'skey'])
    return df    

def read_beta_daily(db, name, start_date=None, end_date=None, skey=None, interval=None, col=None, return_sdi=True): 
    collection = db[name] 
    # Build projection 
    prj = {'_id': 0} 
    if col is not None: 
        if return_sdi: 
            col = ['skey', 'date'] + col 
        for col_name in col: 
            prj[col_name] = 1 
 
    # Build query 
    query = {} 
    if skey is not None: 
        query['skey'] = {'$in': skey} 
    if interval is not None: 
        query['interval'] = {'$in': interval} 
    if start_date is not None: 
        if end_date is not None: 
            query['date'] = {'$gte': start_date, '$lte': end_date} 
        else: 
            query['date'] = {'$gte': start_date} 
    elif end_date is not None: 
        query['date'] = {'$lte': end_date} 
 
    # Load data 
    cur = collection.find(query, prj) 
    df = pd.DataFrame.from_records(cur) 
    if df.empty: 
        df = pd.DataFrame() 
    else: 
        df = df.sort_values(by=['date','skey']) 
    return df  

def build_query(start_date=None, end_date=None, index_id=None):
    query = {}
    def parse_date(x):
        if type(x) == int:
            return x
        elif type(x) == str:
            if len(x) != 8:
                raise Exception("date must be YYYYMMDD format")
            return int(x)
        elif type(x) == datetime.datetime or type(x) == datetime.date:
            return x.strftime("%Y%m%d").astype(int)
        else:
            raise Exception("invalid date type: " + str(type(x)))
    if start_date is not None or end_date is not None:
        query['date'] = {}
        if start_date is not None:
            query['date']['$gte'] = parse_date(start_date)
        if end_date is not None:
            query['date']['$lte'] = parse_date(end_date)
    def parse_symbol(x):
        if type(x) == int:
            return x
        else:
            return int(x)
    if index_id:
        if type(index_id) == list or type(index_id) == tuple:
            query['index_id'] = {'$in': [parse_symbol(x) for x in index_id]}
        else:
            query['index_id'] = parse_symbol(index_id)
    return query

def write_memb_data(db, name, df):
    collection = db[name]
    df1 = []
    for symbol in df['index_id'].unique():
        if symbol in collection.distinct('index_id'):
            symbol = int(symbol)
            m_ax = pd.DataFrame.from_records(collection.find({'index_id':{'$in':[symbol]}}).sort([('date',-1)]).skip(0).limit(1))['date'].values[0]
            df2 = df[(df['index_id'] == symbol) & (df['date'] > m_ax)]
            print(df2)
            df1 += [df2]
        else:
            print(symbol)
            df2 = df[(df['index_id'] == symbol)]
            print(df2)
            df1 += [df2]
    df1 = pd.concat(df1).reset_index(drop=True)
    df1 = df1.to_dict('records')
    collection.insert_many(df1) 

def delete_memb_data(db, name, start_date=None, end_date=None, index_id=None):
    collection = db[name]
    query = build_query(start_date, end_date, index_id)
    if not query:
        print('cannot delete the whole table')
        return None
    collection.delete_many(query)  


database_name = 'com_md_eq_cn'
user = "zhenyuy"
password = "bnONBrzSMGoE"

pd.set_option('max_columns', 200)
db1 = DB("192.168.10.178", database_name, user, password)
IC = read_memb_daily(db1, 'index_memb', 20201120, 20201120, index_id=[1000905])
IF = read_memb_daily(db1, 'index_memb', 20201120, 20201120, index_id=[1000300])
CSI1000 = read_memb_daily(db1, 'index_memb', 20201120, 20201120, index_id=[1000852]) 
CSIRest = read_memb_daily(db1, 'index_memb', 20201120, 20201120, index_id=[1000985]) 

In [4]:
import datetime
datetime.datetime.today().weekday()

0

In [5]:
checkLog = orderLog[(orderLog["updateType"] == 0)]
checkLog = checkLog[checkLog['caamd'] != 0]
checkLog['internal_latency'] = checkLog["clockAtArrival"] - checkLog["caamd"]
# SSE = checkLog[checkLog['secid'] < 2000000]
# SSE["exchange"] = "SH"

# c1 = SSE.groupby(['colo', 'exchange', 'accCode', 'mrstaat', "date"])["internal_latency"].quantile(.95).reset_index().groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["internal_latency"].mean().reset_index()
# c2 = SSE.groupby(['colo', 'exchange', 'accCode', 'mrstaat', "date"])["internal_latency"].median().reset_index().groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["internal_latency"].mean().reset_index()
# c3 = SSE.groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["internal_latency"].count().reset_index()
# c4 = SSE.groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["date"].unique().str.len().reset_index()
# c5 = SSE.groupby(['colo', 'exchange', 'accCode', 'mrstaat', "date"])["internal_latency"].quantile(.95).reset_index().groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["internal_latency"].std().reset_index()

# re1 = pd.merge(c3, c1, on=['colo', 'exchange', 'accCode', 'mrstaat'])
# re1 = re1.rename(columns = {'internal_latency_x': 'count', 'internal_latency_y': '95 percentile'})
# re1 = pd.merge(re1, c2, on=['colo', 'exchange', 'accCode', 'mrstaat'])
# re1 = re1.rename(columns = {'internal_latency': 'median'})
# re1 = pd.merge(re1, c4, on=['colo', 'exchange', 'accCode', 'mrstaat'])
# re1 = pd.merge(re1, c5, on=['colo', 'exchange', 'accCode', 'mrstaat'])
# re1 = re1.rename(columns = {'internal_latency': 'std'})


# # re1 = pd.merge(re1[re1["isMsg"] == 1], re1[re1["isMsg"] == 0], on=["exchange", "colo_account"], how="outer").sort_values(by="median_x").reset_index(drop=True)
# # re2 = pd.merge(re2[re2["isMsg"] == 1], re2[re2["isMsg"] == 0], on=["exchange", "colo_account"], how="outer").sort_values(by="median_y").reset_index(drop=True)
# re = pd.concat([re1]).reset_index(drop=True)


# for col in ['median', '95 percentile']:
#     re[col] = re[col].astype(int)
# for col in ['std']:
#     re[col] = re[col].apply(lambda x: '%.2f'%(x))
    
    
# from IPython.display import display, HTML

# display(HTML(re[re['colo'].str[:5] == 'zt_52'].groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["count", "median", "95 percentile", "std"].first().to_html()))
# display(HTML(re[re['colo'].str[:5] == 'zs_52'].groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["count", "median", "95 percentile", "std"].first().to_html()))

SZE = checkLog[checkLog['secid'] >= 2000000]
SZE["exchange"] = "SZ"

c1 = SZE.groupby(['colo', 'exchange', 'accCode', 'mrstaat', "date"])["internal_latency"].quantile(.95).reset_index().groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["internal_latency"].mean().reset_index()
c2 = SZE.groupby(['colo', 'exchange', 'accCode', 'mrstaat', "date"])["internal_latency"].median().reset_index().groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["internal_latency"].mean().reset_index()
c3 = SZE.groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["internal_latency"].count().reset_index()
c4 = SZE.groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["date"].unique().str.len().reset_index()
c5 = SZE.groupby(['colo', 'exchange', 'accCode', 'mrstaat', "date"])["internal_latency"].quantile(.95).reset_index().groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["internal_latency"].std().reset_index()

re1 = pd.merge(c3, c1, on=['colo', 'exchange', 'accCode', 'mrstaat'])
re1 = re1.rename(columns = {'internal_latency_x': 'count', 'internal_latency_y': '95 percentile'})
re1 = pd.merge(re1, c2, on=['colo', 'exchange', 'accCode', 'mrstaat'])
re1 = re1.rename(columns = {'internal_latency': 'median'})
re1 = pd.merge(re1, c4, on=['colo', 'exchange', 'accCode', 'mrstaat'])
re1 = pd.merge(re1, c5, on=['colo', 'exchange', 'accCode', 'mrstaat'])
re1 = re1.rename(columns = {'internal_latency': 'std'})


# re1 = pd.merge(re1[re1["isMsg"] == 1], re1[re1["isMsg"] == 0], on=["exchange", "colo_account"], how="outer").sort_values(by="median_x").reset_index(drop=True)
# re2 = pd.merge(re2[re2["isMsg"] == 1], re2[re2["isMsg"] == 0], on=["exchange", "colo_account"], how="outer").sort_values(by="median_y").reset_index(drop=True)
re = pd.concat([re1]).reset_index(drop=True)


for col in ['median', '95 percentile']:
    re[col] = re[col].astype(int)
for col in ['std']:
    re[col] = re[col].apply(lambda x: '%.2f'%(x))
    
    
from IPython.display import display, HTML

display(HTML(re[(re['accCode'] == 8856) | (re['accCode'] // 100 == 88)].groupby(['colo', 'exchange', 'accCode', 'mrstaat'])["count", "median", "95 percentile", "std"].first().to_html()))


NameError: name 'orderLog' is not defined

In [56]:
orderLog[orderLog['colo'].isin(['zs_52_06', 'zs_52_08', 'zs_52_09'])]['mrstaat'].unique()

array([1000., 3000.])

In [58]:
pd.set_option('max_rows', 100)
SZE[SZE['accCode'] // 100 == 88].groupby(['colo', 'exchange', 'accCode', 'mrstaat', "date"])["internal_latency"].median()

colo      exchange  accCode  mrstaat  date    
zs_88_04  SZ        8856     1000.0   20201106    36.0
                                      20201109    69.0
                                      20201110    42.0
                                      20201111    68.5
                                      20201112    36.0
                                      20201113    39.5
                                      20201116    40.0
                                      20201117    35.0
                                      20201118    33.5
                                      20201119    49.5
                                      20201120    61.0
                                      20201123    54.0
                             3000.0   20201106    73.0
                                      20201109    68.0
                                      20201110    68.0
                                      20201111    74.0
                                      20201112    67.0
                  

In [64]:
set(['SZ123015', 'SZ123032', 'SZ123042', 'SZ128070', 'SZ123034',
       'SZ123048', 'SZ123061', 'SZ123065', 'SZ123043', 'SZ123045',
       'SZ123052', 'SZ123030', 'SZ123013', 'SH113576', 'SZ127004',
       'SZ123047', 'SZ128073', 'SZ128041', 'SZ123014', 'SZ128103',
       'SZ123012', 'SZ123031', 'SZ123062', 'SH113035', 'SZ128091',
       'SZ123038', 'SH110044', 'SZ128079', 'SZ123057', 'SZ128039',
       'SZ128053', 'SZ123018', 'SZ128052', 'SZ128036', 'SZ127008',
       'SZ128050', 'SZ123027', 'SZ123024', 'SZ123023', 'SZ123029',
       'SH113555', 'SZ123051', 'SZ128076', 'SZ123067', 'SZ123007',
       'SZ128112', 'SZ128108', 'SZ128094', 'SZ123046', 'SH113520']) - \
set(['SZ123015', 'SZ123032', 'SZ128070', 'SZ123042', 'SZ123034',
       'SZ123048', 'SZ123061', 'SH113576', 'SZ123065', 'SZ123030',
       'SZ123045', 'SZ123013', 'SZ127004', 'SZ128103', 'SZ128073',
       'SZ128041', 'SZ123012', 'SZ123062', 'SH110044', 'SZ128079',
       'SZ123052', 'SZ128036', 'SZ123014', 'SZ128091', 'SZ123018',
       'SZ123031', 'SZ123038', 'SH113035', 'SZ128052', 'SZ128053',
       'SZ128039', 'SZ128050', 'SZ123023', 'SH113555', 'SZ127008',
       'SZ123047', 'SZ123051', 'SZ123024', 'SZ123029', 'SZ128076',
       'SZ123026', 'SZ123027', 'SZ128112', 'SH113521', 'SZ123057',
       'SZ128108', 'SH113520', 'SZ123046', 'SH113038', 'SH113011'])

{'SZ123007', 'SZ123043', 'SZ123067', 'SZ128094'}

In [65]:
set(['SZ123015', 'SZ123032', 'SZ128070', 'SZ123042', 'SZ123034',
       'SZ123048', 'SZ123061', 'SH113576', 'SZ123065', 'SZ123030',
       'SZ123045', 'SZ123013', 'SZ127004', 'SZ128103', 'SZ128073',
       'SZ128041', 'SZ123012', 'SZ123062', 'SH110044', 'SZ128079',
       'SZ123052', 'SZ128036', 'SZ123014', 'SZ128091', 'SZ123018',
       'SZ123031', 'SZ123038', 'SH113035', 'SZ128052', 'SZ128053',
       'SZ128039', 'SZ128050', 'SZ123023', 'SH113555', 'SZ127008',
       'SZ123047', 'SZ123051', 'SZ123024', 'SZ123029', 'SZ128076',
       'SZ123026', 'SZ123027', 'SZ128112', 'SH113521', 'SZ123057',
       'SZ128108', 'SH113520', 'SZ123046', 'SH113038', 'SH113011']) - \
set(['SZ123015', 'SZ123032', 'SZ123042', 'SZ128070', 'SZ123034',
       'SZ123048', 'SZ123061', 'SZ123065', 'SZ123043', 'SZ123045',
       'SZ123052', 'SZ123030', 'SZ123013', 'SH113576', 'SZ127004',
       'SZ123047', 'SZ128073', 'SZ128041', 'SZ123014', 'SZ128103',
       'SZ123012', 'SZ123031', 'SZ123062', 'SH113035', 'SZ128091',
       'SZ123038', 'SH110044', 'SZ128079', 'SZ123057', 'SZ128039',
       'SZ128053', 'SZ123018', 'SZ128052', 'SZ128036', 'SZ127008',
       'SZ128050', 'SZ123027', 'SZ123024', 'SZ123023', 'SZ123029',
       'SH113555', 'SZ123051', 'SZ128076', 'SZ123067', 'SZ123007',
       'SZ128112', 'SZ128108', 'SZ128094', 'SZ123046', 'SH113520'])

{'SH113011', 'SH113038', 'SH113521', 'SZ123026'}